In [5]:
import geopandas
import json
import numpy as np
import pandas as pd
import math

In [6]:
infected_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=0&single=true&output=csv", 
                          names=['city', 'infected_in', 'date', 'count', 'age', 'source'], header=0)

infected_df

,city,infected_in,date,count,age,source
0,Скопје,Италија,2020-02-26,1,50-59,https://a1on.mk/macedonia/prv-sluchaj-na-zabol...
1,Дебар,Италија,2020-03-06,2,60-69,http://zdravstvo.gov.mk/dva-novi-potvrdeni-slu...
2,Дебар,Дебар,2020-03-09,2,60-69,https://sitel.com.mk/dva-novi-sluchai-na-koron...
3,Скопје,Италија,2020-03-09,1,50-59,https://makfax.com.mk/makedonija/%D1%84%D0%B8%...
4,Дебар,Дебар,2020-03-09,1,40-49,https://telma.com.mk/ushte-tri-novi-sluchai-na...
5,Дебар,Дебар,2020-03-11,2,60-69,https://a1on.mk/macedonia/dvajca-novi-zaboleni...
6,Дебар,Дебар,2020-03-13,4,50-59,https://civilmedia.mk/13-mart-chetiri-novi-slu...
7,Скопје,Шпанија,3/14/2020,1,непознато,https://www.magazin.mk/%D0%BA%D0%BE%D1%80%D0%B...
8,Дебар,Дебар,3/14/2020,5,непознато,https://makfax.com.mk/makedonija/%d0%b2%d0%b8%...
9,Скопје,Холандија,3/16/2020,1,50-59,https://kurir.mk/makedonija/vesti/pet-novi-slu...


In [7]:
healed_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=757629356&single=true&output=csv",
                       names=['city', 'date', 'count', 'age', 'source'], header=0)
healed_df

,city,date,count,age,source
0,Скопје,2020-03-20,1,50-59,NaN


In [8]:
dead_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1754038661&single=true&output=csv",
                                               header=0, names=['city', 'date', 'count', 'age', 'source'])

dead_df

,city,date,count,age,source


In [14]:
infected_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=0&single=true&output=csv", 
                          names=['city', 'infected_in', 'date', 'count', 'age', 'source'], header=0)

healed_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=757629356&single=true&output=csv",
                       names=['city', 'date', 'count', 'age', 'source'], header=0)

dead_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1754038661&single=true&output=csv",
                                               header=0, names=['city', 'date', 'count', 'age', 'source'])


infected_by_city = infected_df[['city', 'count']].copy().groupby('city').sum()
healded_by_city = healed_df[['city', 'count']].copy().groupby('city').sum()
dead_by_city = dead_df[['city', 'count']].copy().groupby('city').sum()
covid_mk = pd.merge(infected_by_city, healded_by_city, on='city', how='outer', suffixes=['_infected', '_healed'])
covid_mk['count'] = covid_mk.count_infected
covid_mk = pd.merge(covid_mk, dead_by_city, on='city', how='outer', suffixes=['', '_dead'])
covid_mk.fillna(int(0), inplace=True)
covid_mk['count_active'] = covid_mk.count_infected - covid_mk.count_healed - covid_mk.count_dead
covid_mk = covid_mk.astype(int)
covid_mk.drop('count', axis=1, inplace=True)

covid_mk.to_csv('../../opendata/mk/covid19/datasets/summary_by_municipality.csv')
covid_mk

,count_infected,count_healed,count_dead,count_active
city,,,,
Гостивар,1,0,0,1
Дебар,30,0,0,30
Охрид,1,0,0,1
Скопје,37,1,0,36
Штип,2,0,0,2


In [15]:
covid_mk.to_csv('../../opendata/mk/covid19/datasets/summary_by_municipality.csv')

In [16]:
pd.read_csv('../../opendata/mk/covid19/datasets/summary_by_municipality.csv')

,city,count_infected,count_healed,count_dead,count_active
0,Гостивар,1,0,0,1
1,Дебар,30,0,0,30
2,Охрид,1,0,0,1
3,Скопје,37,1,0,36
4,Штип,2,0,0,2


In [19]:
sick_hospitals = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=480309052&single=true&output=csv", header=0, names=['hospital', "date", "count", "age_range", 'origin', 'source'])

recovered_hospitals = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1488187986&single=true&output=csv", header=0, names=['hospital', "date", "count", "age_range", 'source'])

dead_hospitals = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=514503617&single=true&output=csv", header=0, names=['hospital', "date", "count", "age_range", 'source'])

sick_hospital = sick_hospitals.groupby('hospital').sum()
recovered_hospital = recovered_hospitals.groupby('hospital').sum()
dead_hospital = dead_hospitals.groupby('hospital').sum()

covid_hospitals = pd.merge(sick_hospital, recovered_hospital, on='hospital', how='outer', suffixes=['_infected', '_recovered'])
covid_hospitals = pd.merge(covid_hospitals, dead_hospital, on='hospital', how='outer', suffixes=['', '_dead'])
covid_hospitals.fillna(0, inplace=True)
covid_hospitals = pd.merge(covid_hospitals, hospital_data, on='hospital', how='inner')
covid_hospitals = covid_hospitals.set_index("hospital")
covid_hospitals['count_dead'] = covid_hospitals['count']
covid_hospitals.drop(['date', 'age_range', 'count', 'source', 'source_dead'], axis=1, inplace=True)
covid_hospitals['count_active'] = covid_hospitals.count_infected - covid_hospitals.count_recovered - covid_hospitals.count_dead
covid_hospitals['count'] = covid_hospitals.count_active
covid_hospitals['id'] = covid_hospitals.index
covid_hospitals

NameError: name 'hospital_data' is not defined